### This notebook is for using a pretrained keras applications network. Since it has been trained on images with 3 channels, we'll have to use a dataset with 3 channels as well.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

# Load CIFAR10 data

In [ ]:
(Xtrain, ytrain), (Xtest, ytest) = keras.datasets.cifar10.load_data()
Xtrain.shape, ytrain.shape, Xtest.shape, ytest.shape

# Visualize some samples

In [ ]:
n_images = 10
index_sample = np.random.randint(0, Xtrain.shape[0], n_images)

n_columns = 5
n_rows = int(np.ceil(n_images / 5))
fig, axes = plt.subplots(n_rows, n_columns, figsize=(10, 2 * n_rows))

raveled_axes = np.ravel(axes)

for ax, i in zip(raveled_axes, index_sample):
    ax.set_title(ytrain[i], fontsize=16)
    ax.imshow(Xtrain[i])
    
for ax in raveled_axes[n_images:]:
    ax.set_visible(False)

# Prepare data

In [ ]:
def preprocess(df):
    df = df / 255.  # normalize data
    return df

Xtrain = preprocess(Xtrain)
Xtest = preprocess(Xtest)

# Get trained resnet50 from keras

In [ ]:
base_model = keras.applications.resnet.ResNet50(include_top=False, 
                                                weights='imagenet', 
                                                input_shape=Xtrain.shape[1:], 
                                                pooling='avg')
base_model.summary()

In [ ]:
# In the first epochs, we'll not train the weights yet
base_model.trainable = False

# Add a new head to the model

In [ ]:
# add a new head to the neural network
output = keras.layers.Dense(np.unique(ytrain).shape[0], activation='softmax')(base_model.output)

In [ ]:
learning_rate = 0.001

model = keras.models.Model(base_model.input, output)
model.compile(optimizer=keras.optimizers.Adam(learning_rate), 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
# the only trainable variables are the weights and the biases in the last dense layer
len(model.trainable_variables)

# Train 1: fit only the head of the model

In [ ]:
history1 = model.fit(Xtrain, ytrain, batch_size=128, epochs=2, validation_split=0.2)

# Make last couple of layers of pretrained weights trainable

In [ ]:
base_model.trainable = True

In [ ]:
print("Number of layers in the base model: {}".format(len(base_model.layers)))

In [ ]:
# Only finetune the last 30 layers
for layer in base_model.layers[:146]:
    layer.trainable = False

len(model.trainable_variables)

In [ ]:
# compile with smaller learning rate
model.compile(optimizer=keras.optimizers.Adam(learning_rate / 10), 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

# Train 2: finetune the model

In [ ]:
history2 = model.fit(Xtrain, ytrain, batch_size=128, epochs=2, validation_split=0.2)

# Evaluate on test set

In [ ]:
model.evaluate(Xtest, ytest)